*Version 3*<br>
*Last updated: 2021-11-15<p>*
christina.herrick@unh.edu<br>
steeleb@caryinstitute.org<br>
__Please do not distribute__

This colab notebook used deprecated Collection 1 GEE layers. Use with caution. Additionally, there are hard-coded folder paths at use in this Colab notebook. 

# Initial Setup

This notebook uses the [Google Earth Engine](https://developers.google.com/earth-engine) [Python API](https://developers.google.com/earth-engine/guides/python_install) to apply the Single-Channel (SC) algorithm, a model-based algorithm using atmospheric correction coefficients, to Landsat thermal imagery and derives water body skin surface temperatures. 

Algorithms for Landsat 4,5,7 can be found in *Jiménez-Muñoz, J. C., J. Cristóbal, J. A. Sobrino, G. Soria, N. Ninyerola, and X. Pons. 2009. Revision of the Single-Channel Algorithm for Land Surface Temperature Retrieval From Landsat Thermal-Infrared Data. IEEE Transactions on Geoscience and Remote Sensing 47:339–349.* [https://doi.org/10.1109/TGRS.2008.2007125](https://doi.org/10.1109/TGRS.2008.2007125)

Algorithms for Landsat 8 can be found in *Jiménez-Muñoz, J. C., J. A. Sobrino, D. Skoković, C. Mattar, and J. Cristóbal. 2014. Land Surface Temperature Retrieval Methods From Landsat-8 Thermal Infrared Sensor Data. IEEE Geoscience and Remote Sensing Letters 11:1840–1843.* [https://doi.org/10.1109/LGRS.2014.2312032](https://doi.org/10.1109/LGRS.2014.2312032)


To run this script successfully, the bounding box coordinates of a lake are entered in a Google Spreadsheet. If you need help finding those coordinates, try using [OpenStreetMap](https://www.openstreetmap.org/export#map=12/43.3826/-72.0157). The bounding box should be as small as possible while still including the entire lake surface.

## Modules

This section of code blocks imports necessary python modules for the notebook to run. You will be prompted to click one or more URLs and be taken to a page to sign in with your Google account. This account must already be authorized to use Google Earth Engine. If you do not already have access, [fill out this application](https://signup.earthengine.google.com/#!/).

Copy the unique code(s) provided on the web page when prompted and paste where prompted to finish authorization.

In [ ]:
#@markdown __Run this block__ to authorize Colab to authenticate your Google account and give it access to upload files from your local computer ([to be used later](https://colab.research.google.com/drive/1AJFnCB7B7Uev2-0hqIayOb5fkSB0Xn8v#scrollTo=L_Tf54wwIPRy) in the notebook). Once you paste the code, press Enter.
from google.colab import auth, files
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

In [ ]:
#@markdown __Run this block__ to connect colab to your Google Earth Engine account
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
#@markdown Connect Colab to your Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

In [ ]:
#@markdown Run this block to install packages and functions used for interactive mapping (double-click to reveal code)
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from time import strftime, sleep
from datetime import datetime, timedelta
import pytz
import folium
from folium import plugins

# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

print("Packages installed")

## Imported variables

In [ ]:
#@markdown Run this block to import pre-existing features, images, and collections from Earth Engine (double-click to reveal code)
wrs2 = ee.FeatureCollection("users/christinaherrickunh/WRS2_descending_2018")
utmbounds = ee.FeatureCollection("users/christinaherrickunh/UTM_Zone_Boundaries")
sw = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
ag100 = ee.Image("NASA/ASTER_GED/AG100_003")
vapor = ee.ImageCollection("NCEP_RE/surface_wv")
l4t1 = ee.ImageCollection("LANDSAT/LT04/C01/T1")
l4t2 = ee.ImageCollection("LANDSAT/LT04/C01/T2")
l5t1 = ee.ImageCollection("LANDSAT/LT05/C01/T1")
l5t2 = ee.ImageCollection("LANDSAT/LT05/C01/T2")
l7t1 = ee.ImageCollection("LANDSAT/LE07/C01/T1")
l7t2 = ee.ImageCollection("LANDSAT/LE07/C01/T2")
l8t1 = ee.ImageCollection("LANDSAT/LC08/C01/T1")
l8t2 = ee.ImageCollection("LANDSAT/LC08/C01/T2")

print("variables installed")

Below are atmospheric functions derived from modeling the [TIGR2311](https://doi.org/10.1175/1520-0450(2002)041%3C0144:ARNNAF%3E2.0.CO;2) atmospheric sounding database, except Landsat 8, which come from [GAPRI4838](http://dx.doi.org/10.1080/01431161.2015.1054965). These are necessary for the Single-Channel algorithm to run successfully. 

In [ ]:
coeff4 = [0.06674,-0.03447,1.04483,-0.50095,-1.15652,0.09812,-0.04732,1.50453,-0.34405] #  TIGR2311
coeff5 = [0.08158,-0.05707,1.05991,-0.58853,-1.08536,-0.00448,-0.06201,1.59086,-0.33513] #  TIGR2311
coeff7 = [0.06982,-0.03366,1.04896,-0.51041,-1.20026,0.10490,-0.05457,1.52631,-0.32136] #  TIGR2311
coeff8 = [0.04019,0.02916,1.01523,-0.38333,-1.50294,0.20324,0.00918,1.36072,-0.27514] #  GAPRI4838


Landsat scenes contain a QA band meant for bitwise masking of pixels. Information can be found at [USGS](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-1-level-1-quality-assessment-band?qt-science_support_page_related_con=0#qt-science_support_page_related_con). 

Below are lists of binary pixel values used for quality control, represented as `qa_values` (Landsat 4,5,7) and as `qa_val_8` (Landsat 8). Cloud, snow, and ice pixels are removed.

Excel spreadsheets of pixel quality values and their definititions can be found:
- for Landat 4-7 [here](https://unh.box.com/v/landsat4-7-qavalues); 
- for Landsat 8, [here](https://unh.box.com/v/landsat8-qavalues)

In [ ]:
qa_values = ee.List([672,676,680,684,704,708,712,716,928,932,936,940,960,964,968,972]) #  KEEP these pixel values
                        # 1696,1700,1704,1708,1728,1732,1736,1740]) <-- these are snow/ice

qa_val_8 = ee.List([2,2720,2722,2724,2728,2732,2752,2756,2760,2764,2976,2980,2984,2988,3008,3012,3016,3020]) #  KEEP these pixel values
                        # ,3744,3748,3752,3756,3776,3780,3784,3788]); <-- these are snow/ice

## User-set variables

In this section, use either *Option A*, where you define the bounding box, or *Option B* where you import a shape file of your lake. If you are using *Option A*, make sure that the *Option B* section is set to 'no'.

The final section, *Set Remaining Variables*, defines acceptable lake coverage percent, RMSE of Landsat Tier 2 inclusion, dates of interest, and path/row of Landsat flyover.

###Option A: use a bounding box
If you need help finding your coordinates, try using [OpenStreetMap](https://www.openstreetmap.org/export#map=12/43.3826/-72.0157). The bounding box should be as small as possible while still including the entire lake surface.<p>
Enter coordinates to see a map of the bounding box. You can also click on the map to get interactive pop-ups of the latitude (y) and longitude (x), and replace/re-run the coordinates.

In [ ]:
#@markdown `lat1` = North
lat1 = '43.4463'  #@param {type:"string"}
#@markdown `lat2` = South
lat2 = '43.3177'  #@param {type:"string"}
#@markdown `lon1` = West
lon1 = '-72.0882'  #@param {type:"string"}
#@markdown `lon2` = East
lon2 = '-72.0171'  #@param {type:"string"}

use_user_input_file = "no"
lat1 = float(lat1)
lon1 = float(lon1)
lat2 = float(lat2)
lon2 = float(lon2)
box = ee.Geometry.Polygon(
    [[[lon1, lat1], # ur
      [lon2, lat1], # ul
      [lon2, lat2], # ll
      [lon1, lat2]]], None, False) # lr
center_lat = (lat1+lat2)/2
center_lon = (lon1+lon2)/2
locs_map = folium.Map(location=[center_lat,center_lon],zoom_start=10,width=600,height=600)
locs_map.add_child(folium.LatLngPopup())
basemaps['Google Maps'].add_to(locs_map)
locs_map.add_ee_layer(box,{},'aoi')
display(locs_map)

###Option B: upload a table file

Using your Google Earth Engine account, upload a shapefile or csv of your study lake to your Assets and link to it here.<p>![upload_to_ee](https://drive.google.com/thumbnail?id=1lfFoZzQD_wAA7Bnil7mI4zwjKDG5kegh)[click here to enlarge](https://drive.google.com/file/d/1lfFoZzQD_wAA7Bnil7mI4zwjKDG5kegh/view?usp=sharing)<p> ![upload_to_ee](https://drive.google.com/thumbnail?id=1KNaZV63J_LUp6X1OcoLX1wonF0ASiIzi)[click here to enlarge](https://drive.google.com/file/d/1KNaZV63J_LUp6X1OcoLX1wonF0ASiIzi/view?usp=sharing)

In [ ]:
#@markdown File path should begin with 'users/'
user_input_file = "" #@param {type: "string"}
use_user_input_file = "no" #@param ["yes","no"] {allow-input: true}

In [ ]:
shp = ee.FeatureCollection(user_input_file).geometry()
centroid = shp.centroid(10).getInfo()["coordinates"]
center_lat = centroid[1]
center_lon = centroid[0]
print(center_lat)
print(center_lon)

In [ ]:
shp_map = folium.Map(location=[center_lat,center_lon],zoom_start=10,width=600,height=600)
shp_map.add_child(folium.LatLngPopup())
basemaps['Google Maps'].add_to(shp_map)
shp_map.add_ee_layer(shp,{},'aoi')
display(shp_map)

###Set remaining variables

Boundaries of water bodies are automatically 
detected using the JRC Global Surface Water Mapping Layer dataset of percent water occurrence. By default, a pixel has to be classified as water at least 55% of the time. This is defined as `pctTime`

More information on this dataset can be found at [doi:10.1038/nature20584](https://doi.org/10.1038/nature20584)

Landsat scenes from both Tier 1 and Tier 2 of Collection 1 are included for possible use. From Tier 2, only scenes that are processed to L1T with a combined RMS error of no greater than 24 meters are considered.

In [ ]:
pctTime = 55  #@param {type: "number"}
rmse = 24  #@param {type: "number"}

Landsat records begin in 1984 with Landsat 4 TM. Enter the year range that you want to search, as well as months of the year. To search all months, `m1 = 1` and `m2 = 12` <p>
First year (y1) and last year (y2). Years are inclusive.<p>
First month (m1) and last month (m2). Months are inclusive.

In [ ]:
y1 = 1980 #@param {type: "number"}
y2 = 2020 #@param {type: "number"}

m1 =  5#@param {type: "number"}
m2 =  11#@param {type: "number"}
print('Script will search years %d through %d and months %d through %d of each year' % (y1,y2,m1,m2))

In [ ]:
#@markdown This code block uses the extent of Option A/B to calculate the Landsat path(s) and row(s) that it overlaps. It will print as `WRS2 Path/Row: PPPRRR` where `PPP` is the three-digit path, and `RRR` is the three-digit row.
if use_user_input_file=="yes":
  box = shp

pathrow = wrs2.filterBounds(box)
  
num_of_pr = len(pathrow.getInfo()["features"])
path_west = 1
path_east = 233
row_north = 122
row_south = 1

print('landsat path/rows:')
for i in range(0,num_of_pr):
  pr = pathrow.getInfo()["features"][i]["properties"]["PR"]
  p = pr[:3]
  r = pr[3:]
  if int(p) > path_west:
    path_west = int(p)
  if int(p) < path_east:
    path_east = int(p)
  if int(r) < row_north:
    row_north = int(r)
  if int(r) > row_south:
    row_south = int(r)
  print(f"p{p} r{r}")

utm = utmbounds.filterBounds(box)
printUTM = utm.getInfo()["features"][0]["properties"]["ZONE"]
print("UTM Zone:",printUTM, "(does not print more than one zone if more are present)")
crs_out = f"EPSG:326{printUTM}"

zone_map = folium.Map(location=[center_lat,center_lon],zoom_start=7,width=600,height=600)
zone_map.add_child(folium.LatLngPopup())
basemaps['Google Maps'].add_to(zone_map)
zone_map.add_ee_layer(utm,{},'utm zone(s)')
zone_map.add_ee_layer(pathrow,{},'path row')
zone_map.add_ee_layer(box,{},'aoi box')
display(zone_map)
print(f'\nbounding paths: {path_west} to {path_east}')
print(f'bounding rows: {row_north} to {row_south}')

Use the printed statement above to guide your entry for the path and row variables below. If there is only one path and/or one row, enter the same number for `p1,p2` and `r1,r2` <p>
Starting and ending path (*Path 233 starts at the Prime Meridian and decreases moving west-east*)

In [ ]:
#@markdown Use the printed statement above to guide your entry for the path and row variables below. If there is only one path and/or one row, enter the same number for `p1,p2` and `r1,r2` <p> 
#@markdown Starting and ending path (*Path 233 starts at the Prime Meridian and decreases moving west-east*)
p1 = 13 #@param {type: "number"}
p2 =  13#@param {type: "number"}
#@markdown Starting row (*Row 1 starts in the Arctic and increases moving north-south*)
r1 =  30#@param {type: "number"}
r2 = 30 #@param {type: "number"}
print(f"path {p1} to {p2}, row {r1} to {r2}")

# Automatically delineate waterbody

## Find water pixels & boundary

Delineate lake boundaries using [JRC Water dataset](https://global-surface-water.appspot.com/).<p>

In [ ]:
#@markdown This code block reduces the bounding box defined above to the largest water body present, and then counts the 30m pixels within the water body.<p>
#@markdown Landsat scenes included in analysis must have at least 25% of (discontinuous) clear lake pixels.
wateroccurrence = sw.select(0).setDefaultProjection('EPSG:32618')
water = wateroccurrence.gte(pctTime)
water = water.updateMask(water.neq(0))
water = water

def addArea(feature):
  # returns area +/- 1sqm
  return feature.set({'area':feature.geometry().area(1)})  

regions = water.addBands(wateroccurrence).reduceToVectors(
    reducer=ee.Reducer.min(),
    geometry=box,
    scale=30,
    maxPixels=5e9,
    geometryInNativeProjection=False,
    labelProperty='surfaceWater').map(addArea).sort('area',False);

lake_outline = ee.Feature(regions.first())
aoi = lake_outline
geo = lake_outline.geometry()
coords = geo.getInfo()['coordinates'][0]

watercount = water.reduceRegion(reducer=ee.Reducer.count(), 
                                geometry=lake_outline.geometry(),
                                scale=30, bestEffort=True)
totalpixels = watercount.getInfo()["occurrence"]
pixel_min = totalpixels*0.25
print("\n\n# of lake pixels: ", totalpixels);
print("25% of available lake pixels is", pixel_min)

pixel_map = folium.Map(location=[center_lat,center_lon],zoom_start=10, width=600, height=600)
basemaps['Google Satellite'].add_to(pixel_map)
pixel_map.add_ee_layer(geo ,{},'pixels over lake')
display(pixel_map)

## Landsat functions
This block of code defines functions to pre-process Landsat images in preparation for atmospheric correction to water skin temperature.

In [ ]:
def prep4bands(img):
  systime = img.get('system:time_start')
  elev = img.get('SUN_ELEVATION')
  sza = ee.Number(90).subtract(elev)
  uid = img.get('system:index')
  
  radiance = ee.Algorithms.Landsat.calibratedRadiance(img).select(['B6'],['radi'])
  toa = ee.Algorithms.Landsat.TOA(img)
  toa = toa.select(['B[1-6]'],['blue','green','red','nir','swir','temp'])
  
  bt = toa.select(['temp'],['bt']).subtract(273.15)
  
  coeff = ee.Image(coeff4).select([0,1,2,3,4,5,6,7,8],['c11','c12','c13','c21','c22','c23','c31','c32','c33'])
  Bg = ee.Image.constant(1290).select([0],['Bg'])

  #  use the QA band to mask cloudy pixels  
  p_qa = img.select(["BQA"])
  p_mask = p_qa.remap(qa_values,qa_values).mask().int8()
  
  both = radiance.addBands(coeff).addBands(Bg).addBands(bt).updateMask(p_mask).copyProperties(img).set('sza',sza).set('uid',uid).set('system:time_start',systime)
  
  return ee.Image(both)

def prep5bands(img):
  systime = img.get('system:time_start')
  elev = img.get('SUN_ELEVATION')
  sza = ee.Number(90).subtract(elev)
  uid = img.get('system:index')
  
  radiance = ee.Algorithms.Landsat.calibratedRadiance(img).select(['B6'],['radi'])
  toa = ee.Algorithms.Landsat.TOA(img)
  toa = toa.select(['B[1-6]'],['blue','green','red','nir','swir','temp'])
  
  bt = toa.select(['temp'],['bt']).subtract(273.15)
  
  coeff = ee.Image(coeff5).select([0,1,2,3,4,5,6,7,8],['c11','c12','c13','c21','c22','c23','c31','c32','c33'])
  Bg = ee.Image.constant(1256).select([0],['Bg'])

  #  use the QA band to mask cloudy pixels  
  p_qa = img.select(["BQA"])
  p_mask = p_qa.remap(qa_values,qa_values).mask().int8()
  
  both = radiance.addBands(coeff).addBands(Bg).addBands(bt).updateMask(p_mask).copyProperties(img).set('sza',sza).set('uid',uid).set('system:time_start',systime);
  
  return ee.Image(both)

def prep7bands(img):
  systime = img.get('system:time_start')
  elev = img.get('SUN_ELEVATION')
  sza = ee.Number(90).subtract(elev)
  uid = img.get('system:index')
  
  radiance = ee.Algorithms.Landsat.calibratedRadiance(img).select(['B6_VCID_1'],['radi'])
  toa = ee.Algorithms.Landsat.TOA(img)
  toa = toa.select(['B[1-5]','B6_VCID_1'],['blue','green','red','nir','swir','temp'])
  
  bt = toa.select(['temp'],['bt']).subtract(273.15)
  
  coeff = ee.Image(coeff7).select([0,1,2,3,4,5,6,7,8],['c11','c12','c13','c21','c22','c23','c31','c32','c33'])
  Bg = ee.Image.constant(1277).select([0],['Bg'])
  
  # use the QA band to mask cloudy pixels  
  p_qa = img.select(["BQA"])
  p_mask = p_qa.remap(qa_values,qa_values).mask().int8()
  
  both = radiance.addBands(coeff).addBands(Bg).addBands(bt).updateMask(p_mask).copyProperties(img).set('sza',sza).set('uid',uid).set('system:time_start',systime);
  
  return ee.Image(both)

def prep8bands(img):
  systime = img.get('system:time_start')
  elev = img.get('SUN_ELEVATION')
  sza = ee.Number(90).subtract(elev)
  uid = img.get('system:index')
  
  radiance = ee.Algorithms.Landsat.calibratedRadiance(img).select(['B10'],['radi'])
  toa = ee.Algorithms.Landsat.TOA(img)
  toa = toa.select(['B[2-6]','B10'],['blue','green','red','nir','swir','temp'])
  
  bt = toa.select(['temp'],['bt']).subtract(273.15)
  
  coeff = ee.Image(coeff8).select([0,1,2,3,4,5,6,7,8],['c11','c12','c13','c21','c22','c23','c31','c32','c33'])
  Bg = ee.Image.constant(1324).select([0],['Bg'])
  
  # use the QA band to mask cloudy pixels  
  p_qa = img.select(["BQA"]);
  p_mask = p_qa.remap(qa_val_8,qa_val_8).mask().int16()
  both = radiance.addBands(coeff).addBands(Bg).addBands(bt).updateMask(p_mask).copyProperties(img).set('sza',sza).set('uid',uid).set('system:time_start',systime);
  
  return ee.Image(both)

print("Functions imported", strftime("%x %X"))

# Get Skin Surface Temps

In [ ]:
#@title Calculate emissivity
#@markdown Calculate average emissivity values by using ASTER Global Emissivity Dataset (100m)
aster13 = ag100.select(['emissivity_band13']).multiply(0.001).reduceRegion(reducer=ee.Reducer.mean(), geometry=geo, scale=100).get('emissivity_band13')
aster14 = ag100.select(['emissivity_band14']).multiply(0.001).reduceRegion(reducer= ee.Reducer.mean(), geometry= geo, scale=100).get('emissivity_band14')

emissivity = ee.Number.expression('(e13 + e14) / 2', {
  'e13': aster13,
  'e14': aster14
})
e = ee.Number(1).divide(ee.Number(emissivity))
print(f'Average emissivity: {emissivity.getInfo()}')

More information on the emissivity dataset can be found [here](https://lpdaac.usgs.gov/products/ag100bv003/)

## Atmospheric correction function
This defines a function to process prepped Landsat images to water skin temperature.

In [ ]:
def atmosCorr(img):
  img = ee.Image(img)
  systime = img.get('system:time_start')
  crs_out = img.select(0).projection()
  
  radi = img.select('radi')
  bt = img.select('bt')
  Bg = img.select('Bg')
  
  gamma_top = bt.multiply(bt)
  gamma_bot = radi.multiply(Bg)
  gamma = gamma_top.divide(gamma_bot).select([0],['gamma'])
  
  delta_right = gamma_top.divide(Bg)
  delta = bt.subtract(delta_right).select([0],['delta'])
  
  # THIS GETS THE CORRESPONDING VAPOR IMAGE
  v = ee.Image(img.get('vapor')).multiply(0.1).select([0],['vapor'])
  """Since the literature says that the algorithm doesn't work well with
  water vapor columns over 2.5g/cm, those pixels are removed"""
  v = v.mask(v.lte(2.5))
  
  img = img.addBands(v)
  
  psi_1 = img.expression(
    '(c1*v*v)+(c2*v)+c3',{
      'c1': img.select('c11'),
      'c2': img.select('c12'),
      'c3': img.select('c13'),
      'v': img.select('vapor')
    }).select([0],['psi_1'])
    
  psi_2 = img.expression(
    '(c1*v*v)+(c2*v)+c3',{
      'c1': img.select('c21'),
      'c2': img.select('c22'),
      'c3': img.select('c23'),
      'v': img.select('vapor')
    }).select([0],['psi_2'])
  
  psi_3 = img.expression(
    '(c1*v*v)+(c2*v)+c3',{
      'c1': img.select('c31'),
      'c2': img.select('c32'),
      'c3': img.select('c33'),
      'v': img.select('vapor')
    }).select([0],['psi_3'])
 
  surface_temp = psi_1.multiply(radi).add(psi_2).multiply(e).add(psi_3).multiply(gamma).add(delta)
  
  surface_temp = ee.Image(surface_temp).setDefaultProjection(crs_out,None,30)
  surface_temp = ee.Image(surface_temp).select([0],['surface_temp']).copyProperties(img)
  surface_temp = surface_temp.set('system:time_start',systime)

  return surface_temp

print("Function imported:", strftime("%x %X"))

## Filter and Stack Landsat

For each landsat stack, filter by total cloud cover, date, site location, and
make sure all landsat scenes are in descending orbit (wrs<234), then create radiance band, 
toa brightness temp band in Celcius, & band coefficients and constants.
Stack them together, and make sure the metadata and system time carries over. For landsat 8, 
make sure scenes are nadir and the TIRS algorithm isn't preliminary version

In [ ]:
l4 = (l4t1.merge(l4t2).filterMetadata('DATA_TYPE','equals','L1TP') \
      .filterMetadata('GEOMETRIC_RMSE_MODEL','not_greater_than',rmse) \
      .filterBounds(geo) \
      .filter(ee.Filter.calendarRange(y1,y2,'year')) \
      .filter(ee.Filter.calendarRange(m1,m2,'month')) \
      .filterMetadata('WRS_ROW','not_less_than',r1).filterMetadata('WRS_ROW','not_greater_than',r2) \
      .filterMetadata('WRS_PATH','not_greater_than',p1).filterMetadata('WRS_PATH','not_less_than',p2) \
      .map(prep4bands))

l5 = (l5t1.merge(l5t2).filterMetadata('DATA_TYPE','equals','L1TP') \
      .filterMetadata('GEOMETRIC_RMSE_MODEL','not_greater_than',rmse) \
      .filterBounds(geo) \
      .filter(ee.Filter.calendarRange(y1,y2,'year')) \
      .filter(ee.Filter.calendarRange(m1,m2,'month')) \
      .filterMetadata('WRS_ROW','not_less_than',r1).filterMetadata('WRS_ROW','not_greater_than',r2) \
      .filterMetadata('WRS_PATH','not_greater_than',p1).filterMetadata('WRS_PATH','not_less_than',p2) \
      .map(prep5bands))

l7 = (l7t1.merge(l7t2).filterMetadata('DATA_TYPE','equals','L1TP') \
      .filterMetadata('GEOMETRIC_RMSE_MODEL','not_greater_than',rmse) \
      .filterBounds(geo) \
      .filter(ee.Filter.calendarRange(y1,y2,'year')) \
      .filter(ee.Filter.calendarRange(m1,m2,'month')) \
      .filterMetadata('WRS_ROW','not_less_than',r1).filterMetadata('WRS_ROW','not_greater_than',r2) \
      .filterMetadata('WRS_PATH','not_greater_than',p1).filterMetadata('WRS_PATH','not_less_than',p2) \
      .map(prep7bands))

l8 = (l8t1.merge(l8t2).filterMetadata('NADIR_OFFNADIR','equals','NADIR') \
      .filterMetadata('TIRS_SSM_MODEL','not_equals','PRELIMINARY') \
      .filterMetadata('DATA_TYPE','equals','L1TP') \
      .filterMetadata('GEOMETRIC_RMSE_MODEL','not_greater_than',rmse) \
      .filterBounds(geo) \
      .filter(ee.Filter.calendarRange(y1,y2,'year')) \
      .filter(ee.Filter.calendarRange(m1,m2,'month')) \
      .filterMetadata('WRS_ROW','not_less_than',r1).filterMetadata('WRS_ROW','not_greater_than',r2) \
      .filterMetadata('WRS_PATH','not_greater_than',p1).filterMetadata('WRS_PATH','not_less_than',p2) \
      .map(prep8bands))

# filtering by solar zenith angle is useful in high-latitude areas
landsat = ee.ImageCollection((l4).merge(l5).merge(l7).merge(l8)).filterMetadata('sza','less_than',77).sort('system:time_start')
print("Landsat compiled at", strftime("%x %X"))

## Join Landsat scene to water column vapor
This connects the [NCEP/NCAR](https://doi.org/10.1175/1520-0477(1996)077%3C0437:TNYRP%3E2.0.CO;2) water column vapor image to the Landsat image so that there is a value available for atmospheric water vapor during atmospheric correction.<p>
It uses image timestamps to find the vapor image closest in time to Landsat flyover time, and it also uses geometry to find a vapor image that intersects.

In [ ]:
primary = landsat  # primary collection to use
secondary = vapor  # collection used to join to primary
timeDiff = 1000*60*60*6  # maximum time difference of acquisitions in milliseconds (6 hours)
maxDiffFilter = ee.Filter.maxDifference(difference=timeDiff, leftField="system:time_start", rightField="system:time_start")
intersectFilter = ee.Filter.intersects(leftField=".geo",rightField=".geo")  # images must overlap
joinFilter = ee.Filter.And(maxDiffFilter,intersectFilter)
saveFirstJoin = ee.Join.saveBest(matchKey="vapor", measureKey="difference")
joinedVapor = saveFirstJoin.apply(primary,secondary,joinFilter)
print("Landsat and NCEP/NCAR water vapor joined:", strftime("%x %X"))

## Apply atmospheric correction and get skin temp

In [ ]:
temps = joinedVapor.map(atmosCorr)
print("Landsat skin temperatures calculated:", strftime("%x %X"))

Add metadata to each scene that indicates how many visible pixels were included in analysis, and filter images so that only scenes with the minimum number of pixels remain.

In [ ]:
def countPixels(img):
  img = ee.Image(img)
  getCount = img.reduceRegion(**{
      "reducer": ee.Reducer.count(),
      "geometry": geo, 
      "scale": 30})
  count = ee.Dictionary(getCount).get('surface_temp')
  return img.set('pixel_count',count)

countedPixels = temps.map(countPixels).filterMetadata('pixel_count','not_less_than', pixel_min)
print("Landsat filtered:", strftime("%x %X"))

# Prepare data for statistical analysis

## Histogram function
Function that generates a histogram of each image in the collection

In [ ]:
def generate_histogram(img):
  img = ee.Image(img)
  fhisto = img.reduceRegion(**{
      "reducer": ee.Reducer.fixedHistogram(-5,30,350).unweighted(),
      "geometry": geo,
      "scale": 30,
      "maxPixels": 5e9
  })
  return img.set('histogram',fhisto.get('surface_temp'))
print("Will generate histograms with a fixed x-axis between -5 and 30 deg C")

Two functions that convert image pixels to numpy arrays

In [ ]:
def createArrays(img):
  psi_prop = ee.Image(img).sampleRectangle(region=geo, defaultValue=9999)
  getProp = psi_prop.get("psi_1")
  return img.set("img_array", getProp)

def getArrays(client_img_col):
  list_of_arrays = []
  for img in client_img_col:
    prop = img["properties"]["img_array"]
    a = np.array(prop)
    list_of_arrays.append(a)
  return list_of_arrays

In [ ]:
#@markdown This code block executes the above `array` functions on the image collection and convert the image collection from a server-side object to a client-side object. This allows iteration over the image collection with a `for` loop, which provides much more Python functionality.<p>__This step could take some time to run depending on the length of the ImageCollection.__
print("Start:", strftime("%x %X"))
imgs_histo = countedPixels.map(generate_histogram)
generate_arrays = imgs_histo.map(createArrays)

imgCol = generate_arrays.getInfo()["features"]
print("Finish:", strftime("%x %X"))
print("Number of Landsat scenes: ", len(imgCol))

Now that the image collection is client-side, we can get the arrays from each image.

In [ ]:
# returns a list of arrays
imgs_to_arrays = getArrays(imgCol)

## Create single dataframe of all histogram data
This section converts image arrays to a Pandas `DataFrame` for further analysis.

In [ ]:
#@markdown Iterate through the image collection and convert the histogram bins and frequencies to a single Pandas `DataFrame`
length_plots = len(imgCol)
list_of_dfs = []
list_of_uids = []
# fig = make_subplots(rows=length_plots, cols=1)
for i in imgCol:
  uid = (i['properties']['uid']).split("_")[-1]
  list_of_uids.append(uid)
  histogram = i["properties"].get("histogram")

  a = pd.DataFrame(histogram, columns=["bin",uid]).set_index("bin")
  # fig.add_histogram()
  list_of_dfs.append(a)

appended_dfs = pd.concat(list_of_dfs,ignore_index=False, axis=1)
transposed_dfs = appended_dfs.T
print(appended_dfs)

The CSV has the bins (in deg C) in the first column, the image name as the header, and the pixel counts within each bin.<p>
Export the `DataFrame` to a csv and copy it to your Google Drive. You can then go to your Drive and look for a file named *histograms.csv*

In [ ]:
appended_dfs.to_csv('C1_SCA_v2_histograms.csv')
!cp C1_SCA_v2_histograms.csv "drive/My Drive/NASA Landsat Temperature Product/Colab Output/C1_20211115/"
print("file exported to Google Drive")

## View histograms using `matplotlib`

In [ ]:
#@markdown Generate histograms of lake temperature for each Landsat scene.
print(len(list_of_uids)," histograms will be generated")
print("Start:", strftime("%x %X"))
figure, axis = plt.subplots(len(list_of_uids),1, 
                            figsize=(8,4*len(list_of_uids)),frameon=False, dpi=50)
x = appended_dfs.index
for row in range(0,len(list_of_uids)):
  uid = list_of_uids[row]
  y = appended_dfs[uid]
  axis[row].plot(x,y)
  axis[row].set_title(uid)
plt.show()
print("Finish:", strftime("%x %X"))

##Export to CSV

In [ ]:
#@markdown Execute this cell block to export a CSV containing temperature and related image metadata

def exportWholeLakeStats(img):
  img = ee.Image(img).clip(geo)

  landsattime = img.get('system:time_start')
  cloudcover = img.get('CLOUD_COVER')
  vaporImg = ee.Image(img.get('vapor'))
  vaportime = vaporImg.get('system:time_start')
  esd = img.get("EARTH_SUN_DISTANCE")
  elev = img.get('SUN_ELEVATION')
  azi = img.get('SUN_AZIMUTH')
  sza = img.get('sza')
  count = img.get('pixel_count')
  pctAvail = ee.Number(count).divide(totalpixels)

  vaporMath = ee.Algorithms.If(ee.Algorithms.IsEqual(vaporImg,None),-9999,vaporImg)
  vaporMathg = ee.Image(vaporMath).multiply(0.1)

  getWaterCol = vaporMathg.reduceRegion(
      reducer=ee.Reducer.max(),
      geometry=geo,
      bestEffort=True,
      scale=30
      )
  waterCol = ee.Dictionary(getWaterCol).get('pr_wtr')

  stats = img.reduceRegion(
      reducer=ee.Reducer.mean().combine(
          reducer2=ee.Reducer.stdDev(),
          sharedInputs=True).combine(
              reducer2=ee.Reducer.minMax(),
              sharedInputs=True).combine(
                  reducer2 = ee.Reducer.median(),
                      sharedInputs = True).combine(
                          reducer2=ee.Reducer.kurtosis(),
                          sharedInputs = True                    
          ),
    geometry=geo,
    scale=30,
    maxPixels=5e9
  )
    
  
  more_stats = ee.Dictionary({'pixel_count':count,
                              'vapor_time':vaportime,
                              'landsat_time':landsattime,
                              'cloud_cover':cloudcover,
                              'water_column':waterCol,
                              'emiss':emissivity,
                              'elev':elev,
                              'azimuth':azi,
                              'esd':esd,
                              'sza':sza,
                              'pct_lake':pctAvail,
                              'l_exceltime':ee.Number(landsattime).divide(1000.0).divide(86400).add(25569),
                              'v_exceltime': ee.Number(vaportime).divide(1000.0).divide(86400).add(25569)
                              })
  stats2 = ee.Dictionary.combine(stats, more_stats)
  
  return ee.Feature(None,stats2)

temp_stats = countedPixels.map(exportWholeLakeStats)

export_task = ee.batch.Export.table.toDrive(**{
    'collection': temp_stats,
    'description': "C1_SCA_v2_temp_stats",
    "fileFormat": "CSV",
    'folder': "C1_20211115"
})

print("Exporting 'temp_stats.csv'")
export_task.start()
print('Polling for task (id: {}) at'.format(export_task.id))

while export_task.active():
  print(strftime("%x %X"), export_task.status())
  sleep(10)

print("Finished:", strftime("%x %X"))
print('Export should now be visible in Drive.')

# Pair Landsat with *insitu* data
This next section is optional, but allows you to compare any field data you may have to the Landsat data produced here. This section uses the CSV exported above (`"/content/drive/MyDrive/Colab Notebooks/temp_stats.csv"`)<p>
In order for this to run successfully, your data must be in CSV format and have the following headers/columns:


*   `datetime`: Date and time of each temperature reading, formatted as *mm/DD/YYYY HH:MM*
*  `lat_dd`: latitude in decimal degrees
*  `lon_dd`: longitude in decimal degrees
*  `temp_degC`: an integer or float number representing temperature in degrees Celcius
*  `location`: for in-lake statistics, a column with lake zone names (string format, no special characters); can be all the same for a single output or differentiated by lake zones/ sensors



## Upload CSV of *insitu* data
Choose Option 1 or 2. For more ways to import data, see [here](https://colab.research.google.com/notebooks/io.ipynb)  

In [ ]:
#@title Please provide some information about your data
#@markdown What timezone is your data? [Olson Times Wikipedia Page](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones) Specifically, you need to indicate the UTC offset for your data for proper pairing, including whether or not your data timestamp observes Daylight Savings Time. Enter the text for the matching UTC offset and DST behavior from the 'TZ database name' column in the linked table. Note that the sign of the GMT offset is intentionally inverted from the UTC offset.
insitu_timezone = "Etc/GMT+5" #@param {type:"string"}
#@markdown How is your datetime formatted? Please use [strftime](https://strftime.org/) format.
datetime_format = "%Y-%m-%d %H:%M:%S" #@param {type:"string"}

In [ ]:
#@title Option 1: Link to raw CSV from Github
pasted_path = "" #@param {type:"string"}

from dateutil.parser import parse

parser = lambda date: datetime.strptime(date, datetime_format)

is_df = pd.read_csv(pasted_path, parse_dates=[0], date_parser=parser)
is_df["datetime"] = pd.to_datetime(is_df["datetime"])
is_df.set_index('datetime', drop=False, inplace=True)
is_df.index = is_df.index.tz_localize(insitu_timezone).tz_convert("UTC")
print("dataframe created")

In [ ]:
#@title Option 2: Upload data from your local file system to Colab
#@markdown (temporary while connected to current Colab runtime session)
#@markdown Run this cell to upload a file
uploaded = files.upload()
for fn in uploaded.keys():
  print("Paste this in the box below:\n/content/{name}".format(name=fn))

In [ ]:
pasted_path = "/content/insitu_temp_data_v2021-10-20.csv" #@param {type:"string"}

is_df = pd.read_csv(pasted_path)#, parse_dates=[0], date_parser=parser)
#is_df["datetime"] = pd.to_datetime(is_df["datetime"])
#is_df.set_index('datetime', drop=False, inplace=True)
#is_df.index = is_df.index.tz_localize(insitu_timezone).tz_convert("UTC")
print("dataframe created")

In [ ]:
#@title Convert local datetime to UTC time
#@markdown __Run this block__ to apply conversion of your specified local time to UTC time. 
insitu_tz = pytz.timezone(insitu_timezone)
landsat_tz = pytz.timezone("UTC")

def convert_datetime(dt, dtformat=datetime_format):
  #converts string format insitu time to a datetime obj
  dto = datetime.strptime(dt, datetime_format)
  #makes it time aware
  dto_local = insitu_tz.localize(dto)
  #converts to utc
  dto_utc = dto_local.astimezone(landsat_tz)
  return dto_utc

dtobj_series = is_df['datetime']
dtobj_series_conv = dtobj_series.apply(convert_datetime)
is_df['datetime_utc'] = dtobj_series_conv

print("\nDate/time function imported and datetime converted in dataframe", strftime("%x %X"))

## Pair *insitu* data with Landsat data

In [ ]:
#@markdown Enter the window of time (in minutes) from Landsat flyover where *insitu* data should be included. For example, `timewindow = 30` will include any data within 60 minutes of Landsat flyover (+/- 30 minutes)
timewindow = 30 #@param {type:"number"

cwd = '/content/drive/MyDrive/NASA Landsat Temperature Product/Colab Output/C1_20211115'
os.chdir(cwd)

temp_filename = 'C1_SCA_v2_temp_stats'

outfile = ("C1_temp_landsat_paired.csv")


print(f"Time window: +- {timewindow} minutes")
print(f"In-situ time Zone: {insitu_timezone}")
print("\nLandsat input file:\n", os.path.join(cwd,temp_filename+".csv"))
print("\nOutput file will be saved to:\n", os.path.join(cwd,outfile))

min_sec = timewindow * 60

In [ ]:
#@markdown The following codeblock compares the datasets and generate statistics 
#@markdown for in-lake data that matches Landsat flyovers.<br> 
#@markdown It also uses a subfolder called 'ancillary' (and creates the folder if needed) 
#@markdown that keeps a record of any insitu data points that are used for each Landsat scene.
output_dir = '/content/drive/MyDrive/NASA Landsat Temperature Product/Colab Output/C1_20211115'

print("Start:", strftime("%x %X"))
file_name = (output_dir + '/' + temp_filename + '.csv')
gee_csv = pd.read_csv(file_name)
insitu_csv = is_df

#make 'ancillary' folder in defined directory
if not os.path.exists(os.path.join(cwd, 'ancillary')):
    os.makedirs(os.path.join(cwd, 'ancillary'))


#filter GEE output so that it's only as recent as minimum in-situ data
insitumin = min(insitu_csv.datetime_utc)
insitumax = max(insitu_csv.datetime_utc)

def conv_lstime(i):
  return pd.Timestamp(i, unit = 'ms', tz = 'utc')

gee_csv['landsat_time_utc'] = gee_csv.landsat_time.apply(conv_lstime)

gee_overlap = gee_csv[gee_csv.landsat_time_utc>insitumin]
gee_overlap_fin = gee_overlap[gee_overlap.landsat_time_utc<insitumax]
#reindex filtered dataset
gee_overlap_fin = gee_overlap_fin.reset_index(drop=True)

gee_datelist = gee_overlap_fin["landsat_time_utc"]

# function to convert dt_delta to seconds
def delta_tosec(i):
  total_secs = i.seconds + i.days*24*60*60
  return total_secs

for i in range(0,len(gee_datelist)):
  scene = gee_overlap_fin['system:index'][i][-20:]
  landsattime = gee_overlap_fin['landsat_time_utc'][i]

  # create a df of insitu data that are observed within the user-specified cutoff 

  #calculate time delta for each obs
  insitu_csv['dt_delta'] = landsattime - insitu_csv['datetime_utc']
  insitu_csv['delta_secs'] = insitu_csv.dt_delta.apply(delta_tosec)  
  same_time = insitu_csv[(abs(insitu_csv.delta_secs) <= min_sec)]
  
  #if the dataframe is not empty, summarize the results
  if same_time.shape[0]>0:
    print(landsattime)
    gee_overlap_fin.loc[i, "scene"] = scene
    gee_overlap_fin.loc[i, "temp_avg"] = same_time["temp_degC"].mean()
    gee_overlap_fin.loc[i, "t_stdev"] = same_time["temp_degC"].std()
    gee_overlap_fin.loc[i, "depth_avg"] = same_time["depth_m"].mean()
    gee_overlap_fin.loc[i, "d_stdev"] = same_time["depth_m"].std()
    gee_overlap_fin.loc[i, "temp_med"] = same_time["temp_degC"].median()
    gee_overlap_fin.loc[i, "insitu_count"] = same_time.shape[0]

    site_stats = same_time.groupby(['location'])['temp_degC'].agg(['median', 'mean', 'std', 'count'])

    sites = site_stats.axes[0]
    stats = site_stats.axes[1]

    for site in sites:
        for stat in stats:
            newcol = "{0}_{1}".format(str(site), str(stat))
            gee_overlap_fin.loc[i, newcol] = site_stats[stat][site].item()

    if same_time.shape[0] > 0:
        same_time.to_csv(os.path.join(cwd, 'ancillary', scene + ".csv"))

out_csv = gee_overlap_fin[gee_overlap_fin["insitu_count"] > 0]
out_csv.to_csv(os.path.join(cwd, outfile))
("Finished:", strftime("%x %X"))